In [53]:
%cd /mnt/c/Users/user/Desktop/과제/binfo1-work

/mnt/c/Users/user/Desktop/과제/binfo1-work


/root/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
!conda install -y subread

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [7]:
!featureCounts -a gencode.vM27.annotation.gtf -o read-counts.txt *.bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.6

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 10 BAM files                                     ||
||                                                                            ||
||                           CLIP-35L33G.bam                                  ||
||                           CLIP-let7d.bam                                   ||
||                           CLIP-let7f-1.bam                    

In [54]:
import pandas as pd
cnts = pd.read_csv('read-counts.txt', sep='\t', comment='#')
cnts.head()

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam
0,ENSMUSG00000102693.2,chr1,3143476,3144545,+,1070,0,0,0,0,0,0,0,0,0,0
1,ENSMUSG00000064842.3,chr1,3172239,3172348,+,110,0,0,0,0,0,0,0,0,0,0
2,ENSMUSG00000051951.6,chr1;chr1;chr1;chr1;chr1;chr1;chr1,3276124;3276746;3283662;3283832;3284705;349192...,3277540;3277540;3285855;3286567;3287191;349212...,-;-;-;-;-;-;-,6094,4,0,0,0,1,1,1,0,0,0
3,ENSMUSG00000102851.2,chr1,3322980,3323459,+,480,3,0,0,0,0,0,0,0,0,0
4,ENSMUSG00000103377.2,chr1,3435954,3438772,-,2819,0,0,0,0,0,0,0,0,0,0


In [55]:
cnts['clip_enrichment'] = cnts['CLIP-35L33G.bam'] / cnts['RNA-control.bam']
cnts['rden_change'] = (cnts['RPF-siLin28a.bam'] / cnts['RNA-siLin28a.bam']) / (cnts['RPF-siLuc.bam'] / cnts['RNA-siLuc.bam'])
cnts.head()

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam,clip_enrichment,rden_change
0,ENSMUSG00000102693.2,chr1,3143476,3144545,+,1070,0,0,0,0,0,0,0,0,0,0,NaN,NaN
1,ENSMUSG00000064842.3,chr1,3172239,3172348,+,110,0,0,0,0,0,0,0,0,0,0,NaN,NaN
2,ENSMUSG00000051951.6,chr1;chr1;chr1;chr1;chr1;chr1;chr1,3276124;3276746;3283662;3283832;3284705;349192...,3277540;3277540;3285855;3286567;3287191;349212...,-;-;-;-;-;-;-,6094,4,0,0,0,1,1,1,0,0,0,4.0,NaN
3,ENSMUSG00000102851.2,chr1,3322980,3323459,+,480,3,0,0,0,0,0,0,0,0,0,inf,NaN
4,ENSMUSG00000103377.2,chr1,3435954,3438772,-,2819,0,0,0,0,0,0,0,0,0,0,NaN,NaN


In [10]:
#GOterm 찾는 것 부터 내가 수행
!pip install biopython
!pip install bioservices

In [56]:
!pip install biomart
!pip install concurrent

ERROR: Could not find a version that satisfies the requirement concurrent (from versions: none)
ERROR: No matching distribution found for concurrent


In [67]:
'''
#이것도 왠지 모르겠지만 잘 안돌아가서 폐기
from biomart import BiomartServer

def get_go_terms(gene_id):
    # Biomart 서버에 연결
    server = BiomartServer("http://www.ensembl.org/biomart")
    
    # Ensembl 데이터베이스에 액세스하는 데이터셋 선택
    dataset = server.datasets['mmusculus_gene_ensembl']
    
    # 필요한 필드 선택
    fields = ['ensembl_gene_id', 'go_id', 'name_1006']
    
    # 필터 설정: 유전자 ID로 필터링
    filters = {'ensembl_gene_id': gene_id}
    
    # Biomart를 사용하여 데이터 조회
    response = dataset.search({'filters': filters, 'attributes': fields})
    
    # 결과 처리
    go_terms = []
    for row in response.iter_lines():
        row_data = row.decode('utf-8').split('\t')
        go_terms.append((row_data[1], row_data[2]))  # GO ID와 GO 용어를 가져옴
    
    return go_terms
'''

'\n#이것도 왠지 모르겠지만 잘 안돌아가서 폐기\nfrom biomart import BiomartServer\n\ndef get_go_terms(gene_id):\n    # Biomart 서버에 연결\n    server = BiomartServer("http://www.ensembl.org/biomart")\n    \n    # Ensembl 데이터베이스에 액세스하는 데이터셋 선택\n    dataset = server.datasets[\'mmusculus_gene_ensembl\']\n    \n    # 필요한 필드 선택\n    fields = [\'ensembl_gene_id\', \'go_id\', \'name_1006\']\n    \n    # 필터 설정: 유전자 ID로 필터링\n    filters = {\'ensembl_gene_id\': gene_id}\n    \n    # Biomart를 사용하여 데이터 조회\n    response = dataset.search({\'filters\': filters, \'attributes\': fields})\n    \n    # 결과 처리\n    go_terms = []\n    for row in response.iter_lines():\n        row_data = row.decode(\'utf-8\').split(\'\t\')\n        go_terms.append((row_data[1], row_data[2]))  # GO ID와 GO 용어를 가져옴\n    \n    return go_terms\n'

In [57]:
!pip install Bio

In [7]:
from Bio import Entrez

Entrez.email = "your.email@example.com"

def fetch_go_id(go_id):
    handle = Entrez.esearch(db="gene", term=go_id)
    record = Entrez.read(handle)
    handle.close()
    return record

old_go_id = "GO:0005634"
result = fetch_go_id(old_go_id)
print(result)

{'Count': '6810', 'RetMax': '20', 'RetStart': '0', 'IdList': ['100259532', '3503598', '3503593', '3503543', '3503505', '3503443', '3503435', '3503430', '3503396', '3503354', '3503309', '3503305', '3503298', '3503243', '3503139', '3503121', '3503104', '3503007', '3502980', '3502965'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'GO[All Fields]', 'Field': 'All Fields', 'Count': '25065017', 'Explode': 'N'}, {'Term': '0005634[All Fields]', 'Field': 'All Fields', 'Count': '6812', 'Explode': 'N'}, 'AND', 'GROUP'], 'QueryTranslation': 'GO[All Fields] AND 0005634[All Fields]'}


In [46]:
'''
from biomart import BiomartServer

# BioMart 서버에 연결
server = BiomartServer("http://www.ensembl.org/biomart")
dataset = server.datasets['mmusculus_gene_ensembl']

# 사용 가능한 필터 목록 가져오기
filters = dataset.filters

print(filters)
'''

'\nfrom biomart import BiomartServer\n\n# BioMart 서버에 연결\nserver = BiomartServer("http://www.ensembl.org/biomart")\ndataset = server.datasets[\'mmusculus_gene_ensembl\']\n\n# 사용 가능한 필터 목록 가져오기\nfilters = dataset.filters\n\nprint(filters)\n'

In [47]:
'''
from biomart import BiomartServer
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_go_terms_batch(gene_ids, go_terms_filter):
    server = BiomartServer("http://www.ensembl.org/biomart")
    dataset = server.datasets['mmusculus_gene_ensembl']
    fields = ['ensembl_gene_id', 'go_id', 'name_1006']
    filters = {
        'ensembl_gene_id': gene_ids,
        'go': go_terms_filter
    }
    
    response = dataset.search({'filters': filters, 'attributes': fields})
    
    go_terms = []
    for row in response.iter_lines():
        row_data = row.decode('utf-8').split('\t')
        go_terms.append((row_data[0]))#, row_data[1], row_data[2]))  # ensembl_gene_id, go_id, name_1006
    
    return go_terms

def get_go_terms_all(gene_id_list, go_terms_filter, batch_size=100):
    go_terms = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for i in range(0, len(gene_id_list), batch_size):
            batch = gene_id_list[i:i + batch_size]
            futures.append(executor.submit(get_go_terms_batch, batch, go_terms_filter))

        print(futures)
        for future in as_completed(futures):
            try:
                result = future.result()
                go_terms.extend(result)
            except Exception as e:
                print(f"Error: {e}")
    
    return go_terms

# Example usage
gene_id_list = ['ENSMUSG00000064341', 'ENSMUSG00000064342', ...]  # 총 22000개의 gene ID 리스트
specific_go_terms = ['GO:0003677', 'GO:0008150']  # 조회하고 싶은 특정 GO term
go_terms = get_go_terms_all(gene_id_list, specific_go_terms, batch_size=100)

print(go_terms)
'''

'\nfrom biomart import BiomartServer\nfrom concurrent.futures import ThreadPoolExecutor, as_completed\n\ndef get_go_terms_batch(gene_ids, go_terms_filter):\n    server = BiomartServer("http://www.ensembl.org/biomart")\n    dataset = server.datasets[\'mmusculus_gene_ensembl\']\n    fields = [\'ensembl_gene_id\', \'go_id\', \'name_1006\']\n    filters = {\n        \'ensembl_gene_id\': gene_ids,\n        \'go\': go_terms_filter\n    }\n    \n    response = dataset.search({\'filters\': filters, \'attributes\': fields})\n    \n    go_terms = []\n    for row in response.iter_lines():\n        row_data = row.decode(\'utf-8\').split(\'\t\')\n        go_terms.append((row_data[0]))#, row_data[1], row_data[2]))  # ensembl_gene_id, go_id, name_1006\n    \n    return go_terms\n\ndef get_go_terms_all(gene_id_list, go_terms_filter, batch_size=100):\n    go_terms = []\n    with ThreadPoolExecutor(max_workers=10) as executor:\n        futures = []\n        for i in range(0, len(gene_id_list), batch_siz

In [37]:
import gzip

# GZIP 파일 읽기 및 필터링
with gzip.open("goa_mouse.gaf.gz", 'rt') as f:
    # GAF 파일을 데이터프레임으로 읽기
    mouse_data = pd.read_csv(f, sep='\t', comment='!', header=None)

# 열 이름 설정
mouse_data.columns = [
    'DB', 'DB_Object_ID', 'DB_Object_Symbol', 'Qualifier', 'GO_ID',
    'DB:Reference', 'Evidence_Code', 'With_From', 'Aspect', 'DB_Object_Name',
    'DB_Object_Synonym', 'DB_Object_Type', 'Taxon', 'Date', 'Assigned_By',
    'Annotation_Extension', 'Gene_Product_Form_ID'
]

# 데이터 출력 (예: 상위 5개 행)
print(mouse_data.head())

# 필요한 경우 데이터 저장
mouse_data.to_csv('mouse_go_annotations.gaf', sep='\t', index=False)

          DB DB_Object_ID DB_Object_Symbol     Qualifier       GO_ID  \
0  UniProtKB   A0A075B5I2            Trbv4   involved_in  GO:0007166   
1  UniProtKB   A0A075B5I2            Trbv4  is_active_in  GO:0005886   
2  UniProtKB   A0A075B5J4            Trbc2       part_of  GO:0042105   
3  UniProtKB   A0A075B5J6          Gm20730   involved_in  GO:0006955   
4  UniProtKB   A0A075B5J6          Gm20730  is_active_in  GO:0005615   

     DB:Reference Evidence_Code  \
0  GO_REF:0000033           IBA   
1  GO_REF:0000033           IBA   
2  GO_REF:0000033           IBA   
3  GO_REF:0000033           IBA   
4  GO_REF:0000033           IBA   

                                           With_From Aspect  \
0                 MGI:MGI:98608|PANTHER:PTN000588091      P   
1  MGI:MGI:98596|PANTHER:PTN000588091|UniProtKB:A...      C   
2              PANTHER:PTN001511926|UniProtKB:A0A5B9      C   
3  MGI:MGI:98936|PANTHER:PTN000587099|UniProtKB:P...      P   
4              PANTHER:PTN000587099|UniPr

In [58]:
cnts['Geneid'] = cnts['Geneid'].apply(lambda x: x.split('.')[0])

In [59]:
cnts = cnts.dropna(subset=['clip_enrichment'])
cnts = cnts.dropna(subset=['rden_change'])
cnts.head()

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam,clip_enrichment,rden_change
15,ENSMUSG00000102343,chr1;chr1;chr1;chr1;chr1,3975962;4054448;4055383;4055383;4056370,3976357;4054521;4056207;4055574;4056438,-;-;-;-;-,1364,67,0,0,0,31,207,157,1,2,0,2.161290,0.379227
19,ENSMUSG00000025902,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4561154;4561473;4561613;4561936;4562680;456268...,4562891;4562886;4562891;4562891;4563827;456289...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,4772,6,0,0,0,1,8,5,2,3,0,6.000000,0.416667
21,ENSMUSG00000102269,chr1;chr1,4593128;4594669,4593826;4596960,+;+,2991,8,0,0,0,73,28,11,2,9,9,0.109589,0.087302
26,ENSMUSG00000098104,chr1,4758157,4759626,-,1470,29,0,0,0,63,26,15,2,7,0,0.460317,0.164835
30,ENSMUSG00000103922,chr1,4841354,4842422,+,1069,183,0,0,0,95,93,100,42,75,74,1.926316,0.602151


In [66]:
gene_IDs = cnts['Geneid'].tolist()
#print(gene_IDs)
print(len(gene_IDs))

['ENSMUSG00000102343', 'ENSMUSG00000025902', 'ENSMUSG00000102269', 'ENSMUSG00000098104', 'ENSMUSG00000103922', 'ENSMUSG00000033845', 'ENSMUSG00000102275', 'ENSMUSG00000025903', 'ENSMUSG00000033813', 'ENSMUSG00000002459', 'ENSMUSG00000085623', 'ENSMUSG00000033793', 'ENSMUSG00000104046', 'ENSMUSG00000025905', 'ENSMUSG00000033774', 'ENSMUSG00000025907', 'ENSMUSG00000090031', 'ENSMUSG00000103845', 'ENSMUSG00000102135', 'ENSMUSG00000051285', 'ENSMUSG00000098201', 'ENSMUSG00000103509', 'ENSMUSG00000048538', 'ENSMUSG00000097797', 'ENSMUSG00000103067', 'ENSMUSG00000103903', 'ENSMUSG00000025909', 'ENSMUSG00000084353', 'ENSMUSG00000103933', 'ENSMUSG00000101571', 'ENSMUSG00000061024', 'ENSMUSG00000025911', 'ENSMUSG00000025912', 'ENSMUSG00000045210', 'ENSMUSG00000097893', 'ENSMUSG00000025915', 'ENSMUSG00000046334', 'ENSMUSG00000046101', 'ENSMUSG00000098234', 'ENSMUSG00000093178', 'ENSMUSG00000099032', 'ENSMUSG00000025917', 'ENSMUSG00000056763', 'ENSMUSG00000067851', 'ENSMUSG00000042501', 'ENSMUSG0

In [91]:
file_path = 'mouse_go_annotations.gaf'

# GAF 파일을 데이터프레임으로 읽기
gaf_data = pd.read_csv(file_path, sep='\t', comment='!', header=None)

# 열 이름 설정
gaf_data.columns = [
    'DB', 'DB_Object_ID', 'DB_Object_Symbol', 'Qualifier', 'GO_ID',
    'DB:Reference', 'Evidence_Code', 'With_From', 'Aspect', 'DB_Object_Name',
    'DB_Object_Synonym', 'DB_Object_Type', 'Taxon', 'Date', 'Assigned_By',
    'Annotation_Extension', 'Gene_Product_Form_ID'
]

def find_go_id(protein_id, target_GO_ids):
    # 입력된 gene ID에 해당하는 행 찾기
    matching_rows = gaf_data[gaf_data['DB_Object_ID'] == protein_id]
    
    if len(matching_rows) == 0:
        #print("Gene ID not found.")
        return []
    
    # 해당하는 행에서 GO ID 추출
    go_ids = matching_rows['GO_ID'].tolist()
    final_GO = []
    go_ids = list(set(go_ids))
    for i in range(len(go_ids)):
        if go_ids[i] in target_GO_ids:
            final_GO.append(go_ids[i])
    #print(go_ids)
    return final_GO

# Gene ID 입력 받기
#gene_id = input("Enter Gene ID: ")


/tmp/ipykernel_20231/1544218344.py:4: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  gaf_data = pd.read_csv(file_path, sep='\t', comment='!', header=None)


In [62]:
!pip install requests

In [84]:
import requests

def query_biomart(ensembl_gene_ids):
    # Define the BioMart query
    xml_query = f"""
    <!DOCTYPE Query>
    <Query  virtualSchemaName = "default" formatter = "TSV" header = "1" uniqueRows = "1" count = "" datasetConfigVersion = "0.6" >
        <Dataset name = "mmusculus_gene_ensembl" interface = "default" >
            <Filter name = "ensembl_gene_id" value = "{','.join(ensembl_gene_ids)}"/>
            <Attribute name = "ensembl_gene_id" />
            <Attribute name = "uniprotswissprot" />
            <Attribute name = "uniprotsptrembl" />
        </Dataset>
    </Query>
    """
    
    # Send the query to the BioMart service
    response = requests.post('https://www.ensembl.org/biomart/martservice', data={'query': xml_query})
    response.raise_for_status()
    
    # Parse the response
    lines = response.text.strip().split('\n')
    headers = lines[0].split('\t')
    data = [line.split('\t') for line in lines[1:]]
    
    return headers, data

def get_uniprot_ids_for_genes(ensembl_gene_ids):
    headers, data = query_biomart(ensembl_gene_ids)
    
    gene_to_uniprot = {}
    for row in data:
        ensembl_gene_id = row[0]
        uniprot_ids = row[1:]  # Both Swiss-Prot and TrEMBL IDs
        uniprot_ids = [uid for uid in uniprot_ids if uid]  # Remove empty strings
        
        if ensembl_gene_id not in gene_to_uniprot:
            gene_to_uniprot[ensembl_gene_id] = set()
        
        gene_to_uniprot[ensembl_gene_id].update(uniprot_ids)
    
    # Convert sets to lists
    for gene_id in gene_to_uniprot:
        gene_to_uniprot[gene_id] = list(gene_to_uniprot[gene_id])
    
    return gene_to_uniprot

# Example Ensembl Gene IDs
#ensembl_gene_ids = ["ENSMUSG00000025902"]

# Get UniProt IDs for the given Ensembl Gene IDs
#gene_to_uniprot = get_uniprot_ids_for_genes(ensembl_gene_ids)
#print(gene_to_uniprot)

# Print the results
#for gene_id, uniprot_ids in gene_to_uniprot.items():
    #print(f"Gene ID: {gene_id}, UniProt IDs: {uniprot_ids}")


In [70]:
print(get_uniprot_ids_for_genes(gene_IDs))

{'ENSMUSG00000064341': ['Q4JFN6', 'P03888'], 'ENSMUSG00000064345': ['P03893', 'Q9MD59'], 'ENSMUSG00000064351': ['Q9MD68', 'P00397'], 'ENSMUSG00000064354': ['Q7JCZ1', 'P00405'], 'ENSMUSG00000064356': ['Q7JCZ0', 'P03930'], 'ENSMUSG00000064357': ['P00848', 'Q7JCY9'], 'ENSMUSG00000064358': ['P00416', 'Q7JCX7'], 'ENSMUSG00000064360': ['P03899', 'Q7GIP5'], 'ENSMUSG00000065947': ['Q9MD77', 'P03903'], 'ENSMUSG00000064363': ['P03911', 'Q7JCY6'], 'ENSMUSG00000064367': ['Q9MD82', 'P03921'], 'ENSMUSG00000064368': ['P03925', 'Q7JCY4'], 'ENSMUSG00000064370': ['P00158', 'Q7JCZ3'], 'ENSMUSG00000053211': ['P10925'], 'ENSMUSG00000069053': ['A0A9K3AWV9', 'P31254'], 'ENSMUSG00000056673': ['A0A9K3AWU9', 'A0A087WR43', 'Q8BQM3', 'Q62240'], 'ENSMUSG00000069049': ['Q9Z0N2', 'A0A9K3FHH7'], 'ENSMUSG00000068457': ['Q3UQE8', 'E9PWW8', 'D3Z2A3', 'D6RFT4', 'E9Q504', 'G3X9F2', 'F7BBC1', 'D6RJL0'], 'ENSMUSG00000069045': ['A0A087WQV7', 'A0A9K3AWW9', 'Q62095'], 'ENSMUSG00000069044': ['F8VPU6'], 'ENSMUSG00000024304': ['P

In [71]:
G_P_id_dic = get_uniprot_ids_for_genes(gene_IDs)
print(len(G_P_id_dic))

15016


In [93]:
target_GO_terms = ['GO:0005634', 'GO:0031966', 'GO:0016021', 'GO:0005737', 
                   'GO:0016021', 'GO:0005576', 'GO:0009986', 'GO:0005794', 'GO:0005783']
Gene_GO_lists = []
for i in range(len(gene_IDs)):
    uniprot_id = G_P_id_dic.get(gene_IDs[i], [])
    for j in range(len(uniprot_id)):
        x = find_go_id(uniprot_id[j], target_GO_terms)
        if x != []:
            x.insert(0, gene_IDs[i])
            Gene_GO_lists.append(x)
            #print(x)
        
        
        '''
        for h in range(len(target_GO_terms)):
            if target_GO_terms[h] in x:
                print(f'ID : {uniprot_id[j]}, go : {target_GO_terms[h]}')
                '''

In [96]:
a = [item[0] for item in Gene_GO_lists]
b = [item[1:] for item in Gene_GO_lists]

df = pd.DataFrame({'First Column': a, 'Second Column': b})
df.head()

,First Column,Second Column
0,ENSMUSG00000025902,[GO:0005634]
1,ENSMUSG00000025903,"[GO:0005737, GO:0009986, GO:0005783]"
2,ENSMUSG00000033813,[GO:0005634]
3,ENSMUSG00000002459,"[GO:0005634, GO:0005794]"
4,ENSMUSG00000025907,"[GO:0005737, GO:0005634]"


In [40]:
'''
GO_terms = []
for i in range(3):
    a = get_go_terms(gene_IDs[i])
    GO_terms.append(a)

print(GO_terms)
''' 

[[('', '')], [('GO:0010467', 'gene expression'), ('GO:1990830', 'cellular response to leukemia inhibitory factor'), ('GO:0005634', 'nucleus'), ('GO:0043565', 'sequence-specific DNA binding'), ('GO:0010628', 'positive regulation of gene expression'), ('GO:0045944', 'positive regulation of transcription by RNA polymerase II'), ('GO:0005515', 'protein binding'), ('GO:0007283', 'spermatogenesis'), ('GO:2000738', 'positive regulation of stem cell differentiation'), ('GO:0048863', 'stem cell differentiation'), ('GO:0001525', 'angiogenesis'), ('GO:0061009', 'common bile duct development'), ('GO:0048568', 'embryonic organ development'), ('GO:0000976', 'transcription cis-regulatory region binding'), ('GO:0003677', 'DNA binding'), ('GO:0000978', 'RNA polymerase II cis-regulatory region sequence-specific DNA binding'), ('GO:0016055', 'Wnt signaling pathway'), ('GO:0001228', 'DNA-binding transcription activator activity, RNA polymerase II-specific'), ('GO:0006355', 'regulation of DNA-templated tra

In [103]:
'''
target_GO_terms = ['GO:0010467']
go_terms = get_go_terms_all(gene_IDs[1], target_GO_terms, batch_size=100)
print(go_terms)
'''

[<Future at 0x7f95087c8820 state=running>]
['ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00

In [65]:
'''

go_terms = get_go_terms_all(gene_IDs[1], target_GO_terms, batch_size=100)
go_terms = get_go_terms_batch(gene_IDs[1], target_GO_terms)

print(go_terms)
'''

[<Future at 0x7f955ca44e20 state=running>]
[('ENSMUSG00000025902', 'GO:0010467', 'gene expression'), ('ENSMUSG00000025902', 'GO:1990830', 'cellular response to leukemia inhibitory factor'), ('ENSMUSG00000025902', 'GO:0005634', 'nucleus'), ('ENSMUSG00000025902', 'GO:0043565', 'sequence-specific DNA binding'), ('ENSMUSG00000025902', 'GO:0010628', 'positive regulation of gene expression'), ('ENSMUSG00000025902', 'GO:0045944', 'positive regulation of transcription by RNA polymerase II'), ('ENSMUSG00000025902', 'GO:0005515', 'protein binding'), ('ENSMUSG00000025902', 'GO:0007283', 'spermatogenesis'), ('ENSMUSG00000025902', 'GO:2000738', 'positive regulation of stem cell differentiation'), ('ENSMUSG00000025902', 'GO:0048863', 'stem cell differentiation'), ('ENSMUSG00000025902', 'GO:0001525', 'angiogenesis'), ('ENSMUSG00000025902', 'GO:0061009', 'common bile duct development'), ('ENSMUSG00000025902', 'GO:0048568', 'embryonic organ development'), ('ENSMUSG00000025902', 'GO:0000976', 'transcrip

In [ ]:
'''
시간 오래걸려서 폐기 - 심각함 1300분 넘게 소요
GO_terms = []
for i in range(len(gene_IDs)):
    a = get_go_terms(gene_IDs[i])
    GO_terms.append(a)

print(GO_terms)
'''

In [49]:
#2만개 데이터 처리 오래걸려서 중복있나 조회
duplicates = cnts.duplicated(subset=['Geneid'])
print(cnts[duplicates])

Empty DataFrame
Columns: [Geneid, Chr, Start, End, Strand, Length, CLIP-35L33G.bam, CLIP-let7d.bam, CLIP-let7f-1.bam, CLIP-let7g.bam, RNA-control.bam, RNA-siLin28a.bam, RNA-siLuc.bam, RPF-siLin28a.bam, RPF-siLuc.bam, filtered-RPF-siLuc.bam, clip_enrichment, rden_change]
Index: []
